# Demonstração das funcionalidades do Amazon Rekognition

Author: Samir Araújo

https://aws.amazon.com/pt/rekognition/
O Amazon Rekognition é um serviço gerenciado para reconhecimento de imagens. Ele possui diversas funcionalidades como detecção de faces, extração de labels, etc. As suas principais funcionalidades serão apresentadas neste Notebook.

A API em Python boto3 foi utilizada nesta demonstração. Para maiores inforações sobre essa API, consulte:
http://boto3.readthedocs.io/en/latest/reference/services/rekognition.html#Rekognition.Client.detect_labels

In [ ]:
# Make sure you've installed boto3 version 1.4.8
!conda install --yes boto3==1.4.8

In [ ]:
%matplotlib inline

# Importando as bibliotecas necessarias nesta demonstracao
import boto3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw
from IPython.display import Markdown
from notebook import notebookapp

In [ ]:
err_message = "Try shutdown your kernel and start it again. It seems an old version was cached"
assert boto3.__version__ == "1.4.8", err_message

In [ ]:
## Atencao: As credenciais de acesso estao configuradas direto na maquina
## que esta executando este programa. Consulte a documentacao da API (link no cabecalho)
## para voce entender como passar suas credenciais ao cliente 

## Aqui vamos criar o client do Rekognition. 
client = boto3.client('rekognition')
base_path = '/files/workshop/01_Rekognition'

## Definição de funções utilitárias básicas

In [ ]:
# Le uma imagem do disco e a converte para um array de bytes
def load_image(filename):
    with open(filename, "rb") as imageFile:
      f = imageFile.read()
      return bytearray(f)

In [ ]:
# Converte um array para um DataFrame pandas
def convert_to_dataframe(labels):
    data = { 'Label': [], 'Confidence': [] }

    for label in labels:
        data['Label'].append( label['Name'] )
        data['Confidence'].append( label['Confidence'] )

    return pd.DataFrame( data )[ ['Label', 'Confidence'] ]

In [ ]:
# Invoca o Rekognition para extrair as labels de uma imagem
def get_labels(filename):
    response = client.detect_labels(
        Image={'Bytes': load_image(filename)},
        MaxLabels=5,
        MinConfidence=50
    )
    return convert_to_dataframe(response['Labels'])

In [ ]:
# Invoca o Rekognition para extrair as labels de moderacao de uma imagem
def get_moderation_labels(filename):
    response = client.detect_moderation_labels(
        Image={ 'Bytes': load_image(filename) },
        MinConfidence=50
    )
    return convert_to_dataframe(response['ModerationLabels'])

In [ ]:
# Converte uma bounding box retornada para um retangulo
def create_bounding_box(bbox, size):
    if len(bbox) != 4:
        return None
    return [ 
        bbox['Left'] * size[0], bbox['Top'] * size[1],
        (bbox['Left'] * size[0]) + bbox['Width'] * size[0], 
        (bbox['Top'] * size[1]) + bbox['Height'] * size[1]
    ]

In [ ]:
# Invoca o Rekognition para reconhecer celebridades numa foto
def recognize_celebrity(filename):
    response = client.recognize_celebrities(
        Image={'Bytes': load_image(filename)}
    )
    img = Image.open(filename)
    if len(response['CelebrityFaces']) == 0:
        return (None, None, None)
    
    bbox = create_bounding_box( response['CelebrityFaces'][0]['Face']['BoundingBox'], img.size )
    confidence = response['CelebrityFaces'][0]['Face']['Confidence']
    name = response['CelebrityFaces'][0]['Name']
    
    return (name, confidence, bbox )

In [ ]:
# Desenha uma Bounding Box numa foto
def draw_bounding_box(filename, bbox):
    img = Image.open(filename)

    draw = ImageDraw.Draw(img)
    draw.line([(bbox[0], bbox[1]), (bbox[2], bbox[1])], fill='yellow', width=4)
    draw.line([(bbox[2], bbox[1]), (bbox[2], bbox[3])], fill='yellow', width=4)
    draw.line([(bbox[2], bbox[3]), (bbox[0], bbox[3])], fill='yellow', width=4)
    draw.line([(bbox[0], bbox[1]), (bbox[0], bbox[3])], fill='yellow', width=4)
    del draw
    plt.imshow(img)

In [ ]:
# Detecta uma face na foto e suas caracteristicas
def detect_face(filename):
    response = client.detect_faces(
        Image={'Bytes': load_image(filename)},
        Attributes=['ALL']
    )
    if len(response['FaceDetails']) == 0:
        return None
    img = Image.open(filename)
    metadata = response['FaceDetails'][0]
    confidence = metadata['Confidence']
    bbox = create_bounding_box( metadata['BoundingBox'], img.size )
    labels = []
    for label in ('Smile', 'Eyeglasses', 'Sunglasses', 'Gender', 'Beard', 'Mustache', 'EyesOpen', 'MouthOpen'):
        labels.append( {'Name': label + "({})".format( metadata[label]['Value']), 'Confidence': metadata[label]['Confidence'] } )
    for emo in metadata['Emotions']:
        labels.append( {'Name': emo['Type'], 'Confidence': emo['Confidence'] } )
    return ( confidence, bbox, convert_to_dataframe(labels) )

In [ ]:
# Compara a semelhanca entre duas faces
def compare_faces(filenameA, filenameB):
    response = client.compare_faces(
        SourceImage={'Bytes': load_image( filenameA )},
        TargetImage={'Bytes': load_image( filenameB )}
    )
    if len(response['FaceMatches']) == 0:
        return ( None, None, None)
    imgA = Image.open(filenameA)
    imgB = Image.open(filenameB)
    
    similarity = response['FaceMatches'][0]['Similarity']
    bboxA = create_bounding_box( response['SourceImageFace']['BoundingBox'], imgA.size )
    bboxB = create_bounding_box( response['FaceMatches'][0]['Face']['BoundingBox'], imgB.size )
    return (similarity, bboxA, bboxB )

## Detecção de objetos e cenas - client.detect_labels()

In [ ]:
Markdown("![](%s/mosh_thumb.jpg)" % base_path )

In [ ]:
get_labels( 'mosh.jpg' )

In [ ]:
Markdown("![](%s/pandas_thumb.jpg)" % base_path )

In [ ]:
get_labels( 'pandas.jpg' )

In [ ]:
Markdown("![](%s/darth_thumb.jpg)" % base_path )

In [ ]:
get_labels( 'darth.jpg' )

In [ ]:
Markdown("![](%s/robots2_thumb.jpg)" % base_path)

In [ ]:
get_labels( 'robots2.jpg' )

## Detecção de conteúdo sugestivo + moderação - client.detect_moderation_labels()

In [ ]:
Markdown("![](%s/voley_thumb.jpg)" % base_path)

In [ ]:
get_moderation_labels( 'voley.jpg' )

### Agora vamos combinar moderação com detecção de objetos

In [ ]:
labels = pd.concat([ get_moderation_labels( 'voley.jpg' ), get_labels( 'voley.jpg' ) ], ignore_index=True)
labels.sort_values(['Confidence'], ascending=False)

## Reconhecimento de celebridades - client.recognize_celebrities()

In [ ]:
Markdown("![](%s/silvio_thumb.jpg)" % base_path)

In [ ]:
(name, confidence, bbox) = recognize_celebrity( 'silvio.jpg' )
print( name, confidence )
draw_bounding_box('silvio.jpg', bbox )

In [ ]:
Markdown("![](%s/machida_thumb.jpg)"  % base_path)

In [ ]:
(name, confidence, bbox) = recognize_celebrity( 'machida.jpg' )
print( name, confidence )
if bbox: draw_bounding_box('machida.jpg', bbox )

## Detecção de face - client.detect_face()

In [ ]:
Markdown("![](%s/wesley_thumb.png)" % base_path)

In [ ]:
(confidence, bbox, labels) = detect_face('wesley.png')
print( 'Confiança que encontrei uma face: {}'.format( confidence ) )
if bbox: draw_bounding_box('wesley.png', bbox )

In [ ]:
labels

## Comparação de Faces em duas imagens - client.compare_faces()

### Foto de origem

In [ ]:
Markdown("![](%s/ju_paes_thumb.jpg)" % base_path)

### Foto que desejamos procurar o rosto acima

In [ ]:
Markdown("![](%s/ju_we_em_thumb.jpg)" % base_path)

In [ ]:
(similarity, bboxA, bboxB) = compare_faces('ju_paes.jpg', 'ju_we_em.jpg')
print( 'Similarity: {}'.format( similarity ) )

In [ ]:
if bboxA: draw_bounding_box('ju_paes.jpg', bboxA )

In [ ]:
if bboxB: draw_bounding_box('ju_we_em.jpg', bboxB )

## Busca de faces em uma coleção - client.search_faces()
### Collection ID: Funcionarios

In [ ]:
try:
    response = client.delete_collection(
        CollectionId='Funcionarios'
    )
except Exception as e:
    print(e)

In [ ]:
try:
    response = client.create_collection(
        CollectionId='Funcionarios'
    )
    faces = {
        'Glenn': 'coll_glenn.png',
        'Rick': 'coll_rick.jpg',
        'Michonne': 'coll_michonne.jpg',
        'Negan': 'coll_negan.jpg',
        'Turma': 'coll_turma.jpg'
    }
    for ext_id, image_name in faces.items():
        response = client.index_faces(
            CollectionId='Funcionarios',
            Image={ 'Bytes': load_image( image_name ) },
            ExternalImageId=ext_id,
        )
except Exception as e:
    print(e)
    


In [ ]:
faces = {}
bboxes = {}
counter = 1
for face in client.list_faces( CollectionId='Funcionarios', MaxResults=20 )['Faces']:
    if faces.get( face['ExternalImageId'] ) == None: 
        faces[ face['ExternalImageId'] ] = []
    faces[ face['ExternalImageId'] ].append( face['FaceId'] )
    bboxes[ face['ExternalImageId'] ] = face['BoundingBox']
    counter += 1

In [ ]:
print( "FaceId: {}".format( faces['Glenn'][0] ) )
img = Image.open( 'coll_glenn.png' )
draw_bounding_box('coll_glenn.png', create_bounding_box( bboxes['Glenn'], img.size ) )

In [ ]:
print( "FaceId: {}".format( faces['Rick'][0] ) )
img = Image.open( 'coll_rick.jpg' )
draw_bounding_box('coll_rick.jpg', create_bounding_box( bboxes['Rick'], img.size ) )

In [ ]:
print( "FaceId: {}".format( faces['Michonne'][0] ) )
img = Image.open( 'coll_michonne.jpg' )
draw_bounding_box('coll_michonne.jpg', create_bounding_box( bboxes['Michonne'], img.size ) )

In [ ]:
print( "FaceId: {}".format( faces['Negan'][0] ) )
img = Image.open( 'coll_negan.jpg' )
draw_bounding_box('coll_negan.jpg', create_bounding_box( bboxes['Negan'], img.size ) )

In [ ]:
Markdown("![](%s/coll_turma_thumb.jpg)" % base_path)

In [ ]:
face_counter = 1
for faceid in faces['Turma']:
    print( "FaceId{}: {}".format( face_counter, faceid ) )
    face_counter += 1

### Procurando a Michonne na coleção - client.search_faces()

In [ ]:
id_michonne = faces['Michonne'][0]
metadata = client.search_faces(
    CollectionId='Funcionarios',
    FaceId=id_michonne,
    MaxFaces=20
)['FaceMatches'][0]
print( 'Similarity: {}'.format( metadata['Similarity'] ) )
img = Image.open( 'coll_turma.jpg' )
draw_bounding_box('coll_turma.jpg', create_bounding_box( metadata['Face']['BoundingBox'], img.size ) )

### Procurando o Glenn na coleção, através de uma outra foto - client.search_faces_by_image()

In [ ]:
Markdown("![](%s/coll_glenn2_thumb.jpg)" % base_path)

In [ ]:
faces = client.search_faces_by_image(
    CollectionId='Funcionarios',
    Image={ 'Bytes': load_image( 'coll_glenn2.jpg') },
    MaxFaces=20
)['FaceMatches']
print( '# fotos encontradas na coleção: {}'.format( len(faces)))

In [ ]:
print( 'Similarity: {}'.format( faces[0]['Similarity'] ) )
img = Image.open( 'coll_glenn.png' )
draw_bounding_box('coll_glenn.png', create_bounding_box( faces[0]['Face']['BoundingBox'], img.size ) )

In [ ]:
print( 'Similarity: {}'.format( faces[1]['Similarity'] ) )
img = Image.open( 'coll_turma.jpg' )
draw_bounding_box('coll_turma.jpg', create_bounding_box( faces[1]['Face']['BoundingBox'], img.size ) )

# Text in Image client.detect_text()

In [ ]:
Markdown("![](%s/dr_evil.jpg)" % base_path)

In [ ]:
response = client.detect_text(
    Image={ 'Bytes': load_image( 'dr_evil.jpg') },
)
for i in response['TextDetections']:
    print( 'Text[%s] Confidence[%f]' % (i['DetectedText'], i['Confidence']) )

# Video Analysis

Now, right click and 'Save link as' on the following link:
<a href="star_wars.mp4">Star Wars</a><br>
Go to the Rekognition console and upload it to see the new Video Analysis functionality.

# The end :)

In [ ]:
Markdown("![](%s/the_end_thumb.jpg)" % base_path)